In [1]:
# 필요 라이브러리 임포트
import pandas as pd
import mlflow
import mlflow.xgboost
import mlflow.pyfunc
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import roc_auc_score
from mlflow.models import infer_signature
from mlflow.tracking import MlflowClient
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK

# 1. 데이터 로딩 및 전처리
data = pd.read_csv('../data/churn.csv')  # 고객 이탈 데이터 로딩

# Tracking Server 설정
mlflow.set_tracking_uri(uri="http://127.0.0.1:5000") # 환경 변수를 설정했지만, 확실하게 설정

In [2]:

# 사용하지 않는 열 제거 및 feature/target 분리
X = data.drop(['Exited', 'RowNumber', 'CustomerId', 'Surname'], axis=1)
y = data['Exited']

In [3]:
# 범주형 및 수치형 feature 정의
categorical = ['Geography', 'Gender']
numeric = ['CreditScore', 'Age', 'Tenure', 'Balance',
           'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary']


In [4]:
# 전처리 파이프라인: 수치형 표준화 + 범주형 원-핫 인코딩
preprocessor = ColumnTransformer([
    ('num', StandardScaler(), numeric),
    ('cat', OneHotEncoder(), categorical)
])

In [5]:
# 전처리 적용
X_processed = preprocessor.fit_transform(X)



In [6]:
# 학습/테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(
    X_processed, y, test_size=0.2, random_state=42
)

In [8]:

# 2. 하이퍼파라미터 탐색 공간 정의 (Hyperopt 사용)
space = {
    'max_depth': hp.choice('max_depth', range(3, 10)),               # 트리 깊이
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.2),         # 학습률
    'n_estimators': hp.choice('n_estimators', range(50, 200)),       # 트리 개수
    'gamma': hp.uniform('gamma', 0, 5)                                # 가지치기 규제 파라미터
}

In [9]:
# 3. objective 함수 정의: 실험 수행 + 성능 평가 + 조건 충족 시 모델 자동 등록
def objective(params):
    mlflow.set_experiment("practice4")  # 실험 이름 설정
    with mlflow.start_run(nested=True) as run:  # 각 하이퍼파라미터 조합마다 별도 Run 시작
        # 모델 학습
        model = xgb.XGBClassifier(eval_metric='logloss', **params)
        model.fit(X_train, y_train)

        # ROC-AUC 계산을 위한 확률 예측
        probs = model.predict_proba(X_test)[:, 1]
        roc_auc = roc_auc_score(y_test, probs)

        # 실험 결과 로깅 (하이퍼파라미터 및 성능 지표)
        mlflow.log_params(params)
        mlflow.log_metric("roc_auc", roc_auc)

        # 조건 만족 시 (성능 기준 통과), 모델 Registry에 자동 등록 및 Staging 전환
        if roc_auc > 0.86:
            # 모델 시그니처 정의
            signature = infer_signature(X_train, probs)

            # 모델 저장 및 등록
            result = mlflow.xgboost.log_model(
                model,
                artifact_path="model",
                signature=signature,
                registered_model_name="CICDModel"
            )

            # Registry에 등록된 모델 버전 확인 후, Staging 단계로 전환
            client = MlflowClient()
            latest_version = client.get_latest_versions("CICDModel", stages=["None"])[0].version
            client.transition_model_version_stage(
                name="CICDModel",
                version=latest_version,
                stage="Staging",
                archive_existing_versions=True
            )

            print(f"등록 완료: version {latest_version}, AUC={roc_auc:.4f}")

        # Hyperopt가 사용하는 반환 형식 (최소화 대상: -roc_auc)
        return {'loss': -roc_auc, 'status': STATUS_OK}


In [10]:
# 4. 하이퍼파라미터 최적화 수행 (총 30회 시도)
trials = Trials()
best = fmin(
    fn=objective,        # 목적 함수
    space=space,         # 탐색 공간
    algo=tpe.suggest,    # 탐색 알고리즘: TPE
    max_evals=30,        # 총 30회 시도
    trials=trials        # 실험 결과 저장 객체
)

  0%|          | 0/30 [00:00<?, ?trial/s, best loss=?]

2025/10/10 23:42:45 INFO mlflow.tracking.fluent: Experiment with name 'practice4' does not exist. Creating a new experiment.

c:\Users\tmdtn\venvs\mlflow-env\Lib\site-packages\xgboost\sklearn.py:1028: UserWarning: [23:42:46] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\c_api\c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  self.get_booster().save_model(fname)

Successfully registered model 'CICDModel'.
2025/10/10 23:43:16 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: CICDModel, version 1

Created version '1' of model 'CICDModel'.
C:\Users\tmdtn\AppData\Local\Temp\ipykernel_40980\360130756.py:32: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of mode

등록 완료: version 1, AUC=0.8754                          
🏃 View run redolent-squid-846 at: http://127.0.0.1:5000/#/experiments/1/runs/109de31054434a398ebb10071d82c248

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1

  3%|▎         | 1/30 [00:33<16:15, 33.64s/trial, best loss: -0.8754447384296755]

c:\Users\tmdtn\venvs\mlflow-env\Lib\site-packages\xgboost\sklearn.py:1028: UserWarning: [23:43:18] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\c_api\c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  self.get_booster().save_model(fname)

Registered model 'CICDModel' already exists. Creating a new version of this model...
2025/10/10 23:43:48 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: CICDModel, version 2

Created version '2' of model 'CICDModel'.
C:\Users\tmdtn\AppData\Local\Temp\ipykernel_40980\360130756.py:32: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/mode

등록 완료: version 2, AUC=0.8717                                                     
🏃 View run zealous-kit-268 at: http://127.0.0.1:5000/#/experiments/1/runs/fea9056631794f8884140472900766e3

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1                     

  7%|▋         | 2/30 [01:05<15:10, 32.51s/trial, best loss: -0.8754447384296755]

...

등록 완료: version 28, AUC=0.8730                                                     
🏃 View run charming-fish-561 at: http://127.0.0.1:5000/#/experiments/1/runs/839fb56a5ac14bfd99cbb1951b26ee59

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1                      

 93%|█████████▎| 28/30 [10:50<00:38, 19.10s/trial, best loss: -0.8754447384296755]

c:\Users\tmdtn\venvs\mlflow-env\Lib\site-packages\xgboost\sklearn.py:1028: UserWarning: [23:53:35] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\c_api\c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  self.get_booster().save_model(fname)

Registered model 'CICDModel' already exists. Creating a new version of this model...
2025/10/10 23:53:50 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: CICDModel, version 29

Created version '29' of model 'CICDModel'.
C:\Users\tmdtn\AppData\Local\Temp\ipykernel_40980\360130756.py:32: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/mo

등록 완료: version 29, AUC=0.8745                                                     
🏃 View run amazing-dove-151 at: http://127.0.0.1:5000/#/experiments/1/runs/77f2c3990803443d9fa5fee8717977d0

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1                      

 97%|█████████▋| 29/30 [11:06<00:18, 18.12s/trial, best loss: -0.8754447384296755]

c:\Users\tmdtn\venvs\mlflow-env\Lib\site-packages\xgboost\sklearn.py:1028: UserWarning: [23:53:51] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\c_api\c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  self.get_booster().save_model(fname)

Registered model 'CICDModel' already exists. Creating a new version of this model...
2025/10/10 23:54:04 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: CICDModel, version 30

Created version '30' of model 'CICDModel'.
C:\Users\tmdtn\AppData\Local\Temp\ipykernel_40980\360130756.py:32: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/mo

등록 완료: version 30, AUC=0.8719                                                     
🏃 View run funny-midge-383 at: http://127.0.0.1:5000/#/experiments/1/runs/29af4846e7fd41c3b1660e2cb03d4b73

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1                      

100%|██████████| 30/30 [11:21<00:00, 22.71s/trial, best loss: -0.8754447384296755]
